## Step 1: Imports

In [1]:
import pandas as pd
import torch
import matplotlib.pyplot as plt
import numpy as np
import torch
import random
import pickle
from tqdm import tqdm

In [2]:
#set seed for everything
torch.manual_seed(0)
np.random.seed(0)
random.seed(0)

## Step 2: Defining Config

In [3]:
config = {
    "seed"                         : 0,
    "cutoff_date"                  : "2020-01-01",
    "test_cutoff_date"             : "2022-05-01",
    "max_len"                      : 384,
    "batch_size"                   : 1,
    "learning_rate"                : 1e-4,
    "weight_decay"                 : 0.0,
    "mixed_precision"              : "bf16",
    "model_config_path"            : "../working/configs/pairwise.yaml",  # Adjust path as needed
    "epochs"                       : 10,
    "cos_epoch"                    : 5,
    "loss_power_scale"             : 1.0,
    "max_cycles"                   : 1,
    "grad_clip"                    : 0.1,
    "gradient_accumulation_steps"  : 1,
    "d_clamp"                      : 30,
    "max_len_filter"               : 9999999,
    "min_len_filter"               : 10, 
    "structural_violation_epoch"   : 50,
    "balance_weight"               : False,
}

## Step 3 : Get data and do some data processing¶


In [4]:
valid_sequences = pd.read_csv("/kaggle/input/stanford-rna-3d-folding/train_sequences.csv")
valid_labels    = pd.read_csv("/kaggle/input/stanford-rna-3d-folding/train_labels.csv")

valid_sequences.drop(index=[610, 639, 687, 782], inplace=True)
valid_labels.drop(index=[610, 639, 687, 782], inplace=True)

valid_sequences = valid_sequences.reset_index()
valid_labels = valid_labels.reset_index()

In [5]:
valid_labels["pdb_id"] = valid_labels["ID"].apply(lambda x: x.split("_")[0]+'_'+x.split("_")[1])
valid_labels["pdb_id"] 

0         1SCL_A
1         1SCL_A
2         1SCL_A
3         1SCL_A
4         1SCL_A
           ...  
137086    8Z1F_T
137087    8Z1F_T
137088    8Z1F_T
137089    8Z1F_T
137090    8Z1F_T
Name: pdb_id, Length: 137091, dtype: object

### Getting the trainig data ready

In [6]:
import pandas as pd
import os
from tqdm import tqdm

Test = False
# Assuming your CSV files are in a folder
folder_path = '/kaggle/input/rna-dataset-2/extracted'  # Replace with your actual folder path
train_sequences = pd.DataFrame(columns=['target_id', 'sequence', 'temporal_cutoff', 'description', 'all_sequences'])
train_labels    = pd.DataFrame(columns=['ID', 'resname','resid','x_1','y_1','z_1','pdb_id'])

# Process each CSV file in the folder
i = 0
for file in tqdm(os.listdir(folder_path)):
    i += 1
    if Test:
        if i == 100:
            break
    if file.endswith('.csv'):
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path)
        
        # Extract target_id from the first row
        target_id = df['target_id'].iloc[0]
        
        # Create sequence by concatenating all resname values
        sequence = ''.join(df['resname'])
        
        # Add to result dataframe
        new_row = pd.DataFrame({
            'target_id': [target_id],
            'sequence': [sequence],
            'temporal_cutoff': [''],
            'description': [''],
            'all_sequences': ['']
        })
        
        train_sequences = pd.concat([train_sequences, new_row], ignore_index=True)

        # Create labels dataframe entries
        labels_rows = []
        for _, row in df.iterrows():
            # Create ID in format target_id_resid
            id_value = f"{target_id}_{row['resid']}"
            
            labels_rows.append({
                'ID': id_value,
                'resname': row['resname'],
                'resid': row['resid'],
                'x_1': row['x_1'],
                'y_1': row['y_1'],
                'z_1': row['z_1'],
                'pdb_id': target_id
            })
        
        # Add to labels dataframe
        train_labels = pd.concat([train_labels, pd.DataFrame(labels_rows)], ignore_index=True)

# Save the result to a new CSV file
train_sequences.to_csv('converted_data.csv', index=False)
train_labels.to_csv('converted_labels.csv', index=False)

  0%|          | 0/3502 [00:00<?, ?it/s]<ipython-input-6-545765b46205>:56: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  train_labels = pd.concat([train_labels, pd.DataFrame(labels_rows)], ignore_index=True)
100%|██████████| 3502/3502 [01:13<00:00, 47.42it/s]


In [7]:
all_xyz=[]

for pdb_id in tqdm(train_sequences['target_id']):
    df = train_labels[train_labels["pdb_id"]==pdb_id]
    xyz=df[['x_1','y_1','z_1']].to_numpy().astype('float32')
    xyz[xyz<-1e17]=float('Nan');
    all_xyz.append(xyz)
df

  3%|▎         | 96/3502 [00:01<00:57, 59.21it/s]<ipython-input-7-0b47ef48d34d>:6: RuntimeWarning: invalid value encountered in less
  xyz[xyz<-1e17]=float('Nan');
  4%|▎         | 128/3502 [00:02<00:56, 59.79it/s]<ipython-input-7-0b47ef48d34d>:6: RuntimeWarning: invalid value encountered in less
  xyz[xyz<-1e17]=float('Nan');
  4%|▍         | 140/3502 [00:02<00:56, 59.55it/s]<ipython-input-7-0b47ef48d34d>:6: RuntimeWarning: invalid value encountered in less
  xyz[xyz<-1e17]=float('Nan');
  5%|▍         | 158/3502 [00:02<00:56, 59.06it/s]<ipython-input-7-0b47ef48d34d>:6: RuntimeWarning: invalid value encountered in less
  xyz[xyz<-1e17]=float('Nan');
  7%|▋         | 233/3502 [00:03<00:56, 58.11it/s]<ipython-input-7-0b47ef48d34d>:6: RuntimeWarning: invalid value encountered in less
  xyz[xyz<-1e17]=float('Nan');
  8%|▊         | 276/3502 [00:04<00:54, 59.24it/s]<ipython-input-7-0b47ef48d34d>:6: RuntimeWarning: invalid value encountered in less
  xyz[xyz<-1e17]=float('Nan');
  8%|▊     

,ID,resname,resid,x_1,y_1,z_1,pdb_id
261583,1ykq_B_1,G,1,-20.224393,-16.491967,10.122993,1ykq_B
261584,1ykq_B_2,G,2,-19.009405,-18.617505,3.605266,1ykq_B
261585,1ykq_B_3,G,3,-18.749840,-16.821151,-1.208829,1ykq_B
261586,1ykq_B_4,C,4,-19.168544,-14.029909,-5.162431,1ykq_B
261587,1ykq_B_5,G,5,-21.532162,-7.105730,-5.407534,1ykq_B
261588,1ykq_B_6,A,6,-20.545637,-2.526273,-3.256684,1ykq_B
261589,1ykq_B_7,G,7,-17.606030,1.624600,0.147670,1ykq_B
261590,1ykq_B_8,G,8,-17.396982,8.433533,-4.715955,1ykq_B
261591,1ykq_B_9,C,9,-13.165123,12.746040,-5.572929,1ykq_B
261592,1ykq_B_10,C,10,-8.009336,13.896392,-3.604070,1ykq_B


In [8]:
filter_nan = []
max_len    = 0

for xyz in all_xyz:
    if len(xyz) > max_len:
        max_len = len(xyz)
    filter_nan.append((np.isnan(xyz).mean() <= 0.5) & \
                      (len(xyz)<config['max_len_filter']) & \
                      (len(xyz)>config['min_len_filter']))
print(f"Longest sequence in train: {max_len}")

filter_nan      = np.array(filter_nan)
non_nan_indices = np.arange(len(filter_nan))[filter_nan]

train_sequences = train_sequences.loc[non_nan_indices].reset_index(drop=True)
all_xyz         = [all_xyz[i] for i in non_nan_indices]

Longest sequence in train: 960


In [9]:
all_xyz_valid=[]

for pdb_id in tqdm(valid_sequences['target_id']):
    df = valid_labels[valid_labels["pdb_id"]==pdb_id]
    xyz=df[['x_1','y_1','z_1']].to_numpy().astype('float32')
    xyz[xyz<-1e17]=float('Nan');
    all_xyz_valid.append(xyz)

  1%|▏         | 11/840 [00:00<00:07, 107.40it/s]<ipython-input-9-accc2319fea8>:6: RuntimeWarning: invalid value encountered in less
  xyz[xyz<-1e17]=float('Nan');
  4%|▍         | 33/840 [00:00<00:07, 105.91it/s]<ipython-input-9-accc2319fea8>:6: RuntimeWarning: invalid value encountered in less
  xyz[xyz<-1e17]=float('Nan');
<ipython-input-9-accc2319fea8>:6: RuntimeWarning: invalid value encountered in less
  xyz[xyz<-1e17]=float('Nan');
  8%|▊         | 66/840 [00:00<00:07, 106.07it/s]<ipython-input-9-accc2319fea8>:6: RuntimeWarning: invalid value encountered in less
  xyz[xyz<-1e17]=float('Nan');
  9%|▉         | 77/840 [00:00<00:07, 105.39it/s]<ipython-input-9-accc2319fea8>:6: RuntimeWarning: invalid value encountered in less
  xyz[xyz<-1e17]=float('Nan');
<ipython-input-9-accc2319fea8>:6: RuntimeWarning: invalid value encountered in less
  xyz[xyz<-1e17]=float('Nan');
 12%|█▏        | 99/840 [00:00<00:07, 105.42it/s]<ipython-input-9-accc2319fea8>:6: RuntimeWarning: invalid value e

In [10]:
filter_nan_valid = []
max_len_valid    = 0

for xyz in all_xyz_valid:
    if len(xyz) > max_len_valid:
        max_len_valid = len(xyz)
    filter_nan_valid.append((np.isnan(xyz).mean() <= 0.5) & \
                      (len(xyz)<config['max_len_filter']) & \
                      (len(xyz)>config['min_len_filter']))
print(f"Longest sequence in train: {max_len_valid}")

filter_nan_valid      = np.array(filter_nan_valid)
non_nan_indices_valid = np.arange(len(filter_nan_valid))[filter_nan_valid]

valid_sequences = valid_sequences.loc[non_nan_indices_valid].reset_index(drop=True)
all_xyz_valid   = [all_xyz_valid[i] for i in non_nan_indices_valid]

Longest sequence in train: 4298


In [11]:
#pack data into a dictionary

data={
      "sequence":train_sequences['sequence'].to_list(),
      "temporal_cutoff": train_sequences['temporal_cutoff'].to_list(),
      "description": train_sequences['description'].to_list(),
      "all_sequences": train_sequences['all_sequences'].to_list(),
      "xyz": all_xyz
}

In [12]:
#pack data into a dictionary

valid_data={
      "sequence":valid_sequences['sequence'].to_list(),
      "temporal_cutoff": valid_sequences['temporal_cutoff'].to_list(),
      "description": valid_sequences['description'].to_list(),
      "all_sequences": valid_sequences['all_sequences'].to_list(),
      "xyz": all_xyz_valid
}

## Step 4: Split train data into train/val/test¶
We will simply do a temporal split, because that's how testing is done in structural biology in general (in actual blind tests)

In [13]:
# # Split data into train and test
# all_index        = np.arange(len(data['sequence']))
# cutoff_date      = pd.Timestamp(config['cutoff_date'])
# test_cutoff_date = pd.Timestamp(config['test_cutoff_date'])
# train_index      = [i for i, d in enumerate(data['temporal_cutoff']) if pd.Timestamp(d) <= cutoff_date]
# test_index       = [i for i, d in enumerate(data['temporal_cutoff']) if pd.Timestamp(d) > cutoff_date and pd.Timestamp(d) <= test_cutoff_date]

In [14]:
print(f"Train size: {len(train_sequences)}")
print(f"Test size: {len(valid_sequences)}")

Train size: 3412
Test size: 764


## Step 5: Get pytorch dataset¶

In [15]:
from torch.utils.data import Dataset, DataLoader
from ast import literal_eval

def get_ct(bp,s):
    ct_matrix=np.zeros((len(s),len(s)))
    for b in bp:
        ct_matrix[b[0]-1,b[1]-1]=1
    return ct_matrix

class RNA3D_Dataset(Dataset):
    def __init__(self,indices,data):
        self.indices  = indices
        self.data     = data
        self.tokens   = {nt:i for i,nt in enumerate('ACGU')}

    def __len__(self):
        return len(self.indices)
    
    def __getitem__(self, idx):

        idx      = self.indices[idx]
        sequence = [self.tokens[nt] for nt in (self.data['sequence'][idx])]
        sequence = np.array(sequence)
        sequence = torch.tensor(sequence)

        #get C1' xyz
        xyz      = self.data['xyz'][idx]
        xyz      = torch.tensor(np.array(xyz))


        if len(sequence)>config['max_len']:
            crop_start  = np.random.randint(len(sequence)-config['max_len'])
            crop_end    = crop_start+config['max_len']
            sequence    = sequence[crop_start:crop_end]
            xyz         = xyz[crop_start:crop_end]
            
        return {'sequence' : sequence, 'xyz' : xyz}

In [16]:
train_dataset = RNA3D_Dataset(train_sequences.index,data)
val_dataset   = RNA3D_Dataset(valid_sequences.index,valid_data)

In [17]:
# for i in range(len(val_dataset)):
#     try:
#         length = len(val_dataset[i]['sequence'])
#     except Exception as e:
#         print(i,e)

In [18]:
def filter_mismatched_samples(dataset):
    valid_indices = []
    
    for i in range(len(dataset)):
        sample = dataset[i]
        # Check if the first dimension of sequence matches the first dimension of xyz
        if sample['sequence'].shape[0] == sample['xyz'].shape[0]:
            valid_indices.append(i)
        else:
            print(f"Removing sample {i}: sequence shape {sample['sequence'].shape}, xyz shape {sample['xyz'].shape}")
    
    # Create a filtered dataset using a subset of the original dataset
    from torch.utils.data import Subset
    filtered_dataset = Subset(dataset, valid_indices)
    
    print(f"Original dataset size: {len(dataset)}")
    print(f"Filtered dataset size: {len(filtered_dataset)}")
    
    return filtered_dataset

val_dataset = filter_mismatched_samples(val_dataset)


Removing sample 26: sequence shape torch.Size([13]), xyz shape torch.Size([12, 3])
Removing sample 27: sequence shape torch.Size([29]), xyz shape torch.Size([28, 3])
Removing sample 30: sequence shape torch.Size([19]), xyz shape torch.Size([18, 3])
Removing sample 34: sequence shape torch.Size([44]), xyz shape torch.Size([43, 3])
Original dataset size: 764
Filtered dataset size: 760


In [19]:
train_loader = DataLoader(train_dataset,batch_size=1,shuffle=True)
val_loader   = DataLoader(val_dataset,batch_size=1,shuffle=False)

## Step 6: Get RibonanzaNet¶
We will add a linear layer to predict xyz of C1' atoms

In [20]:
! pip install einops

In [21]:
import sys

sys.path.append("/kaggle/input/ribonanzanet2d-final")


from Network import *
import yaml

class Config:
    def __init__(self, **entries):
        self.__dict__.update(entries)
        self.entries=entries

    def print(self):
        print(self.entries)

def load_config_from_yaml(file_path):
    with open(file_path, 'r') as file:
        config = yaml.safe_load(file)
    return Config(**config)

class finetuned_RibonanzaNet(RibonanzaNet):
    def __init__(self, config, pretrained=False):
        config.dropout=0.1
        super(finetuned_RibonanzaNet, self).__init__(config)
        #self.ct_predictor=nn.Linear(64,1)
        if pretrained:
            self.load_state_dict(torch.load("/kaggle/input/ribonanzanet-weights/RibonanzaNet.pt",map_location='cpu'))
        
        self.dropout=nn.Dropout(0.0)
        self.xyz_predictor=nn.Linear(256,3)

    def forward(self,src):
        
        #with torch.no_grad():
        sequence_features, pairwise_features=self.get_embeddings(src, torch.ones_like(src).long().to(src.device))
        xyz=self.xyz_predictor(sequence_features)

        return xyz

In [22]:
model = finetuned_RibonanzaNet(load_config_from_yaml("/kaggle/input/ribonanzanet2d-final/configs/pairwise.yaml"),pretrained=True).cuda()

constructing 9 ConvTransformerEncoderLayers


<ipython-input-21-e7f7b6f52864>:28: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state_dict(torch.load("/kaggle/input/ribonanzanet-weights/RibonanzaNet.pt",map_lo

## Step 7: Training loop¶
we will use dRMSD loss on the predicted xyz. the loss function is invariant to translations, rotations, and reflections. because dRMSD is invariant to reflections, it cannot distinguish chiral structures, so there may be better loss functions

In [23]:
def calculate_distance_matrix(X,Y,epsilon=1e-4):
    return (torch.square(X[:,None]-Y[None,:])+epsilon).sum(-1).sqrt()


def dRMSD(pred_x,
          pred_y,
          gt_x,
          gt_y,
          epsilon=1e-4,Z=10,d_clamp=None):
    pred_dm=calculate_distance_matrix(pred_x,pred_y)
    gt_dm=calculate_distance_matrix(gt_x,gt_y)

    mask=~torch.isnan(gt_dm)
    mask[torch.eye(mask.shape[0]).bool()]=False

    if d_clamp is not None:
        rmsd=(torch.square(pred_dm[mask]-gt_dm[mask])+epsilon).clip(0,d_clamp**2)
    else:
        rmsd=torch.square(pred_dm[mask]-gt_dm[mask])+epsilon

    return rmsd.sqrt().mean()/Z

def local_dRMSD(pred_x,
          pred_y,
          gt_x,
          gt_y,
          epsilon=1e-4,Z=10,d_clamp=30):
    pred_dm=calculate_distance_matrix(pred_x,pred_y)
    gt_dm=calculate_distance_matrix(gt_x,gt_y)



    mask=(~torch.isnan(gt_dm))*(gt_dm<d_clamp)
    mask[torch.eye(mask.shape[0]).bool()]=False



    rmsd=torch.square(pred_dm[mask]-gt_dm[mask])+epsilon
    # rmsd=(torch.square(pred_dm[mask]-gt_dm[mask])+epsilon).sqrt()/Z
    #rmsd=torch.abs(pred_dm[mask]-gt_dm[mask])/Z
    return rmsd.sqrt().mean()/Z

def dRMAE(pred_x,
          pred_y,
          gt_x,
          gt_y,
          epsilon=1e-4,Z=10,d_clamp=None):
    flag = False
    pred_dm=calculate_distance_matrix(pred_x,pred_y)
    gt_dm=calculate_distance_matrix(gt_x,gt_y)


    if pred_dm.shape[0] != gt_dm.shape[0]:
        print(f"Got it")
        print(pred_dm.shape)
        print(gt_dm.shape)
        flag = True
        
    mask=~torch.isnan(gt_dm)
    
    mask[torch.eye(mask.shape[0]).bool()]=False

    rmsd=torch.abs(pred_dm[mask]-gt_dm[mask])

    return rmsd.mean()/Z, flag

import torch

def align_svd_mae(input, target, Z=10):
    """
    Aligns the input (Nx3) to target (Nx3) using SVD-based Procrustes alignment
    and computes RMSD loss.
    
    Args:
        input (torch.Tensor): Nx3 tensor representing the input points.
        target (torch.Tensor): Nx3 tensor representing the target points.
    
    Returns:
        aligned_input (torch.Tensor): Nx3 aligned input.
        rmsd_loss (torch.Tensor): RMSD loss.
    """
    assert input.shape == target.shape, "Input and target must have the same shape"

    #mask 
    mask=~torch.isnan(target.sum(-1))

    input=input[mask]
    target=target[mask]
    
    # Compute centroids
    centroid_input = input.mean(dim=0, keepdim=True)
    centroid_target = target.mean(dim=0, keepdim=True)

    # Center the points
    input_centered = input - centroid_input.detach()
    target_centered = target - centroid_target

    # Compute covariance matrix
    cov_matrix = input_centered.T @ target_centered

    # SVD to find optimal rotation
    U, S, Vt = torch.svd(cov_matrix)

    # Compute rotation matrix
    R = Vt @ U.T

    # Ensure a proper rotation (det(R) = 1, no reflection)
    if torch.det(R) < 0:
        Vt[-1, :] *= -1
        R = Vt @ U.T

    # Rotate input
    aligned_input = (input_centered @ R.T.detach()) + centroid_target.detach()
    
    return torch.abs(aligned_input-target).mean()/Z

In [24]:
from tqdm import tqdm
from torch.amp import GradScaler

if Test:
    epochs=1
    cos_epoch=1
else:
    epochs=50
    cos_epoch=35


best_loss=np.inf
optimizer = torch.optim.Adam(model.parameters(), weight_decay=0.0, lr=0.0001) #no weight decay following AF

batch_size=1

#for cycle in range(2):

criterion=torch.nn.BCEWithLogitsLoss(reduction='none')

scaler = GradScaler()

schedule=torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=(epochs-cos_epoch)*len(train_loader)//batch_size)

best_val_loss=99999999999
for epoch in range(epochs):
    model.train()
    tbar=tqdm(train_loader)
    total_loss=0
    oom=0
    for idx, batch in enumerate(tbar):
        #try:
        sequence=batch['sequence'].cuda()
        gt_xyz=batch['xyz'].cuda().squeeze()

        #with torch.autocast(device_type='cuda', dtype=torch.float16):
        pred_xyz=model(sequence).squeeze()
        
        loss1, flag=dRMAE(pred_xyz,pred_xyz,gt_xyz,gt_xyz) 
        loss = loss1 + align_svd_mae(pred_xyz, gt_xyz)
             #local_dRMSD(pred_xyz,pred_xyz,gt_xyz,gt_xyz)

        if flag:
            print(f"Error Sequence : {sequence, gt_xyz}")
            
        if loss!=loss:
            stop

        
        (loss/batch_size).backward()

        if (idx+1)%batch_size==0 or idx+1 == len(tbar):

            torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
            optimizer.step()
            optimizer.zero_grad()
            
            if (epoch+1)>cos_epoch:
                schedule.step()
      
        total_loss+=loss.item()
        
        tbar.set_description(f"Epoch {epoch + 1} Loss: {total_loss/(idx+1)} OOMs: {oom}")

    tbar=tqdm(val_loader)
    model.eval()
    val_preds=[]
    val_loss=0
    for idx, batch in enumerate(tbar):
        sequence=batch['sequence'].cuda()
        gt_xyz=batch['xyz'].cuda().squeeze()

        with torch.no_grad():
            pred_xyz=model(sequence).squeeze()
            loss, flag=dRMAE(pred_xyz,pred_xyz,gt_xyz,gt_xyz)
            
        val_loss+=loss.item()
        val_preds.append([gt_xyz.cpu().numpy(),pred_xyz.cpu().numpy()])
    val_loss=val_loss/len(tbar)
    print(f"val loss: {val_loss}")
    
    
    
    if val_loss<best_val_loss:
        best_val_loss=val_loss
        best_preds=val_preds
        torch.save(model.state_dict(),'RibonanzaNet-3D.pt')

    # 1.053595052265986 train loss after epoch 0
torch.save(model.state_dict(),'RibonanzaNet-3D-final.pt')

  0%|          | 0/3412 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
  0%|          | 0/760 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
100%|██████████| 760/760 [00:45<00:00, 16.83it/s]


val loss: 1.0122510364377184


100%|██████████| 760/760 [00:45<00:00, 16.80it/s]


val loss: 0.825936627623282


100%|██████████| 760/760 [00:45<00:00, 16.77it/s]


...

val loss: 0.40950466965227145


100%|██████████| 760/760 [00:45<00:00, 16.67it/s]


val loss: 0.4032145107017928


100%|██████████| 760/760 [00:45<00:00, 16.69it/s]


val loss: 0.40128159113963574
